In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Przygotowanie klasy Images odpowiedzialnej za utworzenie obiektu przetrzymującego właściwości wszystkich palet, obrazów w skali szarości oraz binaryzacji w celu późniejszego łatwiejszego ich wyświetlania

In [ ]:
class Images:
    def __init__(self, srcImg, gtImg):
        self.Image = srcImg
        self.GT_Image = cv2.cvtColor(gtImg, cv2.COLOR_BGR2GRAY)

        self.RGB_Image = cv2.cvtColor(srcImg, cv2.COLOR_BGR2RGB)
        self.XYZ_Image = cv2.cvtColor(srcImg, cv2.COLOR_BGR2XYZ)
        self.LAB_Image = cv2.cvtColor(srcImg, cv2.COLOR_BGR2LAB)
        self.HLS_Image = cv2.cvtColor(srcImg, cv2.COLOR_BGR2HLS)
        self.HSV_Image = cv2.cvtColor(srcImg, cv2.COLOR_BGR2HSV)

        self.RGB_Gray = cv2.cvtColor(self.RGB_Image, cv2.COLOR_RGB2GRAY)
        self.XYZ_Gray = self.XYZ_Image[:, :, 1]
        self.LAB_Gray = self.LAB_Image[:, :, 0]
        self.HLS_Gray = self.HLS_Image[:, :, 1]
        self.HSV_Gray = self.HSV_Image[:, :, 2]

        (RGBThr, self.RGBBin) = cv2.threshold(self.RGB_Gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        (XYZThr, self.XYZBin) = cv2.threshold(self.XYZ_Gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        (LABThr, self.LABBin) = cv2.threshold(self.LAB_Gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        (HLSThr, self.HLSBin) = cv2.threshold(self.HLS_Gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        (HSVThr, self.HSVBin) = cv2.threshold(self.HSV_Gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        self.RGB_Niblack = cv2.ximgproc.niBlackThreshold(self.RGB_Gray, 255, cv2.THRESH_BINARY, 199, -0.2,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_NIBLACK)
        self.RGB_Sauvola = cv2.ximgproc.niBlackThreshold(self.RGB_Gray, 255, cv2.THRESH_BINARY, 33, 0.5,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_SAUVOLA)

        self.XYZ_Niblack = cv2.ximgproc.niBlackThreshold(self.XYZ_Gray, 255, cv2.THRESH_BINARY, 199, -0.2,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_NIBLACK)
        self.XYZ_Sauvola = cv2.ximgproc.niBlackThreshold(self.XYZ_Gray, 255, cv2.THRESH_BINARY, 33, 0.5,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_SAUVOLA)

        self.LAB_Niblack = cv2.ximgproc.niBlackThreshold(self.LAB_Gray, 255, cv2.THRESH_BINARY, 199, -0.2,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_NIBLACK)
        self.LAB_Sauvola = cv2.ximgproc.niBlackThreshold(self.LAB_Gray, 255, cv2.THRESH_BINARY, 33, 0.5,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_SAUVOLA)

        self.HLS_Niblack = cv2.ximgproc.niBlackThreshold(self.HLS_Gray, 255, cv2.THRESH_BINARY, 199, -0.2,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_NIBLACK)
        self.HLS_Sauvola = cv2.ximgproc.niBlackThreshold(self.HLS_Gray, 255, cv2.THRESH_BINARY, 33, 0.5,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_SAUVOLA)

        self.HSV_Niblack = cv2.ximgproc.niBlackThreshold(self.HSV_Gray, 255, cv2.THRESH_BINARY, 199, -0.2,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_NIBLACK)
        self.HSV_Sauvola = cv2.ximgproc.niBlackThreshold(self.HSV_Gray, 255, cv2.THRESH_BINARY, 33, 0.5,
                                                         binarizationMethod=cv2.ximgproc.BINARIZATION_SAUVOLA)

        self.AllColors = [self.RGB_Image, self.XYZ_Image, self.LAB_Image, self.HLS_Image, self.HSV_Image]
        self.Grays = [self.RGB_Gray, self.XYZ_Gray, self.LAB_Gray, self.HLS_Gray, self.HSV_Gray]
        self.Bins = [self.RGBBin, self.XYZBin, self.LABBin, self.HLSBin, self.HLSBin, self.HSVBin]
        self.Niblacks = [self.RGB_Niblack, self.XYZ_Niblack, self.LAB_Niblack, self.HLS_Niblack, self.HSV_Niblack]
        self.Sauvolas = [self.RGB_Sauvola, self.XYZ_Sauvola, self.LAB_Sauvola, self.HLS_Sauvola, self.HSV_Sauvola]
        self.colorNames = ["RGB", "XYZ", "LAB", "HLS", "HSV"]


def Precision(u, u0_GT):
    temp_tp = (u == 0) & (u0_GT == 0)
    temp_fp = (u == 0) & (u0_GT != 0)
    count_tp = sum(sum(temp_tp))
    count_fp = sum(sum(temp_fp))
    temp_p = count_tp / (count_fp + count_tp)
    return temp_p


def Recall(u, u0_GT):
    temp_tp = (u == 0) & (u0_GT == 0)
    temp_fn = (u != 0) & (u0_GT == 0)
    count_tp = sum(sum(temp_tp))
    count_fn = sum(sum(temp_fn))
    temp_r = count_tp / (count_fn + count_tp)
    return temp_r


def F_measure(u, u0_GT):
    temp_p = Precision(u, u0_GT)
    temp_r = Recall(u, u0_GT)
    temp_f = 100 * 2 * (temp_p * temp_r) / (temp_p + temp_r)
    return temp_f


def PSNR(u, u0_GT):
    (xm, ym) = size(u)
    temp_fp = (u == 0) & (u0_GT != 0)
    temp_fn = (u != 0) & (u0_GT == 0)
    err=sum(sum(temp_fp | temp_fn)) / (xm * ym)
    return


# Wczytanie 3 różnych obrazów z bazy dibco jako argumentów do konstruktora obiektu z klasy Images

Obrazy zostają zamknięte w tablicy `ARGUMENTS` w celu wyświetlania obiektów w pętli

In [ ]:
Im1 = Images(cv2.imread("./DIBCO/H04.png"), cv2.imread("./DIBCO/H04_GT.tif"))
Im2 = Images(cv2.imread("./DIBCO/H08.png"), cv2.imread("./DIBCO/H08_GT.tif"))
Im3 = Images(cv2.imread("./DIBCO/H12.png"), cv2.imread("./DIBCO/H12_GT.tif"))

# Zadanie nr 1 - binaryzacja obrazu DIBCO metodą OTSU dla różnych metod konwersji obrazu kolorowego do skali szarośc

In [ ]:
ARGUMENTS = [Im1, Im2, Im3]

for argIm in ARGUMENTS:
    for i in range(5):
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        fig.set_figheight(15)
        fig.set_figwidth(15)

        ax1.axis('off')
        ax1.imshow(argIm.AllColors[i])
        ax1.set_title(argIm.colorNames[i])

        ax2.axis('off')
        ax2.imshow(argIm.Grays[i], cmap='gray', vmin=0, vmax=255)
        ax2.set_title(argIm.colorNames[i] + ' Gray')

        ax3.axis('off')
        ax3.imshow(argIm.Bins[i], cmap='gray', vmin=0, vmax=255)
        ax3.set_title(argIm.colorNames[i] + ' OTSU')

# Zadanie nr 2 -  implementacja metody Sauvola oraz Niblack, porównanie z metodą OTSU

In [ ]:
for argIm in ARGUMENTS:
    for i in range(5):
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        fig.set_figheight(15)
        fig.set_figwidth(15)

        ax1.axis('off')
        ax1.imshow(argIm.Bins[i], cmap='gray', vmin=0, vmax=255)
        ax1.set_title(argIm.colorNames[i] + ' OTSU')

        ax2.axis('off')
        ax2.imshow(argIm.Niblacks[i], cmap='gray', vmin=0, vmax=255)
        ax2.set_title(argIm.colorNames[i] + ' Niblack')

        ax3.axis('off')
        ax3.imshow(argIm.Sauvolas[i], cmap='gray', vmin=0, vmax=255)
        ax3.set_title(argIm.colorNames[i] + ' Sauvola')